In [45]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df = pd.read_csv('train.csv')

df.dropna(axis=0)
df.set_index('id', inplace = True)

df.head()

,text,author
id,,
id26305,"This process, however, afforded me no means of...",EAP
id17569,It never once occurred to me that the fumbling...,HPL
id11008,"In his left hand was a gold snuff box, from wh...",EAP
id27763,How lovely is spring As we looked from Windsor...,MWS
id12958,"Finding nothing else, not even gold, the Super...",HPL


In [46]:
import nltk
nltk.download('brown')
nltk.download('twitter_samples')
nltk.download('pos_tag_sents')
nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import stopwords
from nltk.tag import pos_tag_sents
from nltk.tokenize import word_tokenize


stopWords = set(stopwords.words('english'))

#creating a function to encapsulate preprocessing, to mkae it easy to replicate on  submission data
def processing(df):
    #lowering and removing punctuation
    df['processed'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]','', x.lower()))\
  
    #tagging all the words and converting to list
    df['tagged'] = pos_tag_sents(df['processed'].apply(word_tokenize).tolist())\
    
    

    #total length of sentence
    df['length'] = df['processed'].apply(lambda x: len(x))
    #get number of words
    df['words'] = df['processed'].apply(lambda x: len(x.split(' ')))
    df['words_not_stopword'] = df['processed'].apply(lambda x: len([t for t in x.split(' ') if t not in stopWords]))
    #get the average word length
    df['avg_word_length'] = df['processed'].apply(lambda x: np.mean([len(t) for t in x.split(' ') if t not in stopWords]) if len([len(t) for t in x.split(' ') if t not in stopWords]) > 0 else 0)
    #get the average word length
    df['commas'] = df['text'].apply(lambda x: x.count(','))
    
    #count the number of adjectives
    df['adjectives'] = df['tagged'].apply(lambda x: len([t for t in x if t[1] == 'JJ']))\
        
    #count the number of nouns
    df['noun'] = df['tagged'].apply(lambda x: len([t for t in x if t[1] == 'NN']))\
    
    #count the number of verbs
    df['verbs'] = df['tagged'].apply(lambda x: len([t for t in x if t[1] == 'VB']))\

    return(df)

df = processing(df)

df.head()

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\nusan\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\nusan\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Error loading pos_tag_sents: Package 'pos_tag_sents' not
[nltk_data]     found in index
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nusan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text,author,processed,tagged,length,words,words_not_stopword,avg_word_length,commas,adjectives,noun,verbs
id,,,,,,,,,,,,
id26305,"This process, however, afforded me no means of...",EAP,this process however afforded me no means of a...,"[(this, DT), (process, NN), (however, RB), (af...",224,41,21,6.380952,4,2,10,1
id17569,It never once occurred to me that the fumbling...,HPL,it never once occurred to me that the fumbling...,"[(it, PRP), (never, RB), (once, RB), (occurred...",70,14,6,6.166667,0,1,2,1
id11008,"In his left hand was a gold snuff box, from wh...",EAP,in his left hand was a gold snuff box from whi...,"[(in, IN), (his, PRP$), (left, JJ), (hand, NN)...",195,36,19,5.947368,4,4,9,0
id27763,How lovely is spring As we looked from Windsor...,MWS,how lovely is spring as we looked from windsor...,"[(how, WRB), (lovely, RB), (is, VBZ), (spring,...",202,34,21,6.476190,3,5,6,0
id12958,"Finding nothing else, not even gold, the Super...",HPL,finding nothing else not even gold the superin...,"[(finding, VBG), (nothing, NN), (else, RB), (n...",170,27,16,7.187500,2,1,5,1


In [47]:
#splitting train and test model
from sklearn.model_selection import train_test_split

features= [c for c in df.columns.values if c  not in ['id','text','author']]
numeric_features= [c for c in df.columns.values if c  not in ['id','text','author','processed']]
target = 'author'

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.33, random_state=42)
X_train.head()

,processed,tagged,length,words,words_not_stopword,avg_word_length,commas,adjectives,noun,verbs
id,,,,,,,,,,
id19417,this panorama is indeed glorious and i should ...,"[(this, DT), (panorama, NN), (is, VBZ), (indee...",91,18,6,6.666667,1,1,4,1
id09522,there was a simple natural earnestness about h...,"[(there, EX), (was, VBD), (a, DT), (simple, JJ...",240,44,18,6.277778,4,6,6,0
id22732,who are you pray that i duc de lomelette princ...,"[(who, WP), (are, VBP), (you, PRP), (pray, IN)...",387,74,38,5.552632,9,1,15,4
id10351,he had gone in the carriage to the nearest tow...,"[(he, PRP), (had, VBD), (gone, VBN), (in, IN),...",118,24,11,5.363636,0,0,6,0
id24580,there is no method in their proceedings beyond...,"[(there, EX), (is, VBZ), (no, DT), (method, NN...",71,13,5,7.000000,1,0,3,0


In [48]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

In [49]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

text = Pipeline([
                ('selector', TextSelector(key='processed')),
                ('tfidf', TfidfVectorizer( stop_words='english'))
            ])

text.fit_transform(X_train)

<13117x21516 sparse matrix of type '<class 'numpy.float64'>'
	with 148061 stored elements in Compressed Sparse Row format>

In [50]:
from sklearn.preprocessing import StandardScaler

length =  Pipeline([
                ('selector', NumberSelector(key='length')),
                ('standard', StandardScaler())
            ])

length.fit_transform(X_train)

array([[-0.50769254],
       [ 0.88000324],
       [ 2.24907223],
       ...,
       [-0.46112557],
       [-0.14447015],
       [-0.39593181]])

In [51]:
words =  Pipeline([
                ('selector', NumberSelector(key='words')),
                ('standard', StandardScaler())
            ])
words_not_stopword =  Pipeline([
                ('selector', NumberSelector(key='words_not_stopword')),
                ('standard', StandardScaler())
            ])
avg_word_length =  Pipeline([
                ('selector', NumberSelector(key='avg_word_length')),
                ('standard', StandardScaler())
            ])
commas =  Pipeline([
                ('selector', NumberSelector(key='commas')),
                ('standard', StandardScaler()),
            ])

In [52]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion([('text', text), 
                      ('length', length),
                      ('words', words),
                      ('words_not_stopword', words_not_stopword),
                      ('avg_word_length', avg_word_length),
                      ('commas', commas)])

feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(X_train)

<13117x21521 sparse matrix of type '<class 'numpy.float64'>'
	with 213646 stored elements in Compressed Sparse Row format>

In [66]:
#Uses Logistic Regression instead of Random Forest classifier.
from sklearn.ensemble import RandomForestClassifier

from inspect import classify_class_attrs
from sklearn.linear_model import LogisticRegression


pipeline = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression( random_state = 42)),
])

pipeline.fit(X_train, y_train)

preds = pipeline.predict(X_test)
np.mean(preds == y_test)

#print the detailed evaluation report (prec, recall, f1 etc.)
from sklearn.metrics import classification_report
classify_class = classification_report(y_test, preds)
print(classify_class)


c:\Users\nusan\anaconda3\envs\nlpenv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

         EAP       0.74      0.85      0.79      2587
         HPL       0.81      0.74      0.77      1852
         MWS       0.83      0.74      0.78      2023

    accuracy                           0.78      6462
   macro avg       0.79      0.77      0.78      6462
weighted avg       0.79      0.78      0.78      6462



In [54]:
pipeline.get_params().keys()


dict_keys(['memory', 'steps', 'verbose', 'features', 'classifier', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text', 'features__length', 'features__words', 'features__words_not_stopword', 'features__avg_word_length', 'features__commas', 'features__text__memory', 'features__text__steps', 'features__text__verbose', 'features__text__selector', 'features__text__tfidf', 'features__text__selector__key', 'features__text__tfidf__analyzer', 'features__text__tfidf__binary', 'features__text__tfidf__decode_error', 'features__text__tfidf__dtype', 'features__text__tfidf__encoding', 'features__text__tfidf__input', 'features__text__tfidf__lowercase', 'features__text__tfidf__max_df', 'features__text__tfidf__max_features', 'features__text__tfidf__min_df', 'features__text__tfidf__ngram_range', 'features__text__tfidf__norm', 'features__text__tfidf__preprocessor', 'features__text__tfidf__smooth_idf', 'features__text__tfidf__stop_words'

In [70]:
from sklearn.model_selection import GridSearchCV

hyperparameters = { 'features__text__tfidf__max_df': [0.9, 0.95],
                    'features__text__tfidf__ngram_range': [(1,1), (1,2)],
                   'classifier__max_depth': [50, 70],
                    'classifier__min_samples_leaf': [1,2]
                  }
clf = GridSearchCV(pipeline, hyperparameters, cv=5)
 
# Fit and tune model
clf.fit(X_train, y_train)

ValueError: Invalid parameter 'max_depth' for estimator LogisticRegression(random_state=42). Valid parameters are: ['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'].

In [40]:
clf.best_params_


NameError: name 'clf' is not defined

In [71]:
#refitting on entire training data using best settings
clf.refit

preds = clf.predict(X_test)
probs = clf.predict_proba(X_test)

np.mean(preds == y_test)

#print the detailed evaluation report (prec, recall, f1 etc.)
from sklearn.metrics import classification_report
classify_class = classification_report(y_test, preds)

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
submission = pd.read_csv('test.csv')

#preprocessing
submission = processing(submission)
predictions = clf.predict_proba(submission)

preds = pd.DataFrame(data=predictions, columns = clf.best_estimator_.named_steps['classifier'].classes_)

#generating a submission file
result = pd.concat([submission[['id']], preds], axis=1)
result.set_index('id', inplace = True)
result.head()

,EAP,HPL,MWS
id,,,
id02310,0.282536,0.260946,0.456518
id24541,0.581561,0.189180,0.229259
id00134,0.341303,0.449904,0.208792
id27757,0.528043,0.232676,0.239282
id04081,0.552280,0.196810,0.250910
